# Verification of Thesis Results

## Imports 

In [1]:
import sys
import os
import pathlib
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm import tqdm

from src.models.baseline.kmeans import runKmeans
from tensorflow.keras.datasets import cifar100, cifar10, mnist

from src.models.baseline.helper import *

## Required Paths

In [2]:
cwd = Path(os.getcwd())
root = cwd.parent.parent
data = pathlib.PurePath(root, 'data')
history = pathlib.PurePath(data, 'history')
interim = pathlib.PurePath(data, 'interim')
res = pathlib.PurePath(data, 'results', 'thesis')

## Parameters and Datasets

In [10]:
"""
test_param_grid ={
    1 : {
        'K' : [50, 100, 200, 500, 1000],
        'epsilon' : 0.01
    },
    2 : {
        'K' : 1000,
        'epsilon' : [0.01, 0.05, 0.1, 1.0]
    }
}
"""
test_param_grid ={
    1 : {
        'K' : [500],
        'epsilon' : 0.01
    },
    2 : {
        'K' : 1000,
        'epsilon' : [0.01, 0.05, 0.1, 1.0]
    }
}
model_param_grid = {
    'MNIST' : {
        'batch_size' : 1000,
        'epochs' : 15,
        'save_history' : True,
        'path' : history
    },
    'CIFAR10' : {
        'batch_size' : 64,
        'epochs' : 30,
        'save_history' : True,
        'path' : history
    },
    'CIFAR100' : {
        'batch_size' : 64,
        'epochs' : 30,
        'save_history' : True,
        'path' : history
    }
}

In [11]:
datasets = {
    'MNIST' : {
        'data' : dataset_normalize(mnist.load_data()),
        'shape' : (28, 28, 1)
        },
    'CIFAR10' : {
       'data' : dataset_normalize(cifar10.load_data()),
       'shape' : (32, 32, 3) 
    },
    'CIFAR100' : {
        'data' : dataset_normalize(cifar100.load_data()),
       'shape' : (32, 32, 3) 
    }
}

In [12]:
datasets =  {
    'CIFAR100' : {
        'data' : dataset_normalize(cifar100.load_data()),
       'shape' : (32, 32, 3) 
    }
}

In [13]:
partition_dir = interim
seed = 8008

## Strategy 1 (Variable K)

In [14]:
for key, v in datasets.items():
    
    intermediate = []

    x_train = v['data']['x_train']
    x_test = v['data']['x_test']
    y_train = v['data']['y_train']
    y_test = v['data']['y_test']

    print("Running Test 1 on {}...".format(key))

    for k in tqdm(test_param_grid[1]['K']):
        dir = pathlib.PurePath(partition_dir, key + str(k) + '_partitions.tsv')
        if Path(dir.as_posix()).exists():
            print("Loading Partitions for {} dataset with {} clusters".format(key, k))
            with open(dir) as f:
                lines = f.readlines()
            lines = [line.rstrip() for line in lines]
            x = []
            y = []
            for line in lines:
                tokens = line.split()
                x_vec = np.zeros(len(tokens)-1)
                for i in range(len(tokens)-1):
                    x_vec[i] = float(tokens[i])

                x.append(x_vec)
                y.append(int(tokens[-1]))
        else:
            print("Generating Partitions for {} dataset with {} clusters".format(key, k))
            x_vecs = flatten(x_train)
            x, y = partition(x_vecs, k, SEED=seed, write_path=pathlib.PurePath(interim, key + str(k) + '_partitions.tsv'))

        kmeans = runKmeans(k, (x_train, x_test), (y_train, y_test), v['shape'], key, model_param_grid[key])
        results = runTest(k, test_param_grid[1]['epsilon'], (x_train, x_test), (y_train, y_test), (x, y), v['shape'], model_param_grid[key], key, (False, True, True))
        
        sets = [kmeans, results['epsilon'], results['complete']]
        for set in sets:
            set['K'] = k
            intermediate.append(set)

    metrics = pd.DataFrame(intermediate)
    metrics.to_csv(pathlib.PurePath(res, 'strategy1_500_1000_{}.csv'.format(key)))
    print("Test 1 Completed Successfully for {}".format(key))

Running Test 1 on CIFAR100...


  0%|          | 0/1 [00:00<?, ?it/s]

Loading Partitions for CIFAR100 dataset with 500 clusters
y_train shape: (50000,)


d:\CONDA\envs\tfgpu\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy on K-Means : 0.0968
Accuracy on Epsilon_Neighbourhood : 0.0967


100%|██████████| 1/1 [19:07<00:00, 1147.45s/it]

Accuracy on Complete_Information : 0.439
Test 1 Completed Successfully for CIFAR100


## Strategy 2 (Variable $\epsilon$)

In [7]:
for key, v in datasets.items():

    intermediate = []

    x_train = v['data']['x_train']
    x_test = v['data']['x_test']
    y_train = v['data']['y_train']
    y_test = v['data']['y_test']

    k = test_param_grid[2]['K']
    dir = pathlib.PurePath(partition_dir, key + str(k) + '_partitions.tsv')
    
    if Path(dir.as_posix()).exists():
        print("Loading Partitions for {} dataset with {} clusters".format(key, k))
        with open(dir) as f:
            lines = f.readlines()
        lines = [line.rstrip() for line in lines]
        x = []
        y = []
        for line in lines:
            tokens = line.split()
            x_vec = np.zeros(len(tokens)-1)
            for i in range(len(tokens)-1):
                x_vec[i] = float(tokens[i])

            x.append(x_vec)
            y.append(int(tokens[-1]))
    else: 
        print("Generating Partitions for {} dataset with {} clusters".format(key, k))
        x_vecs = flatten(x_train)
        x, y = partition(x_vecs, k, SEED=seed, write_path=pathlib.PurePath(interim, key + str(k) + '_partitions.tsv'))

    print("Running Test 2 on {}...".format(key))

    for e in tqdm(test_param_grid[2]['epsilon']): 
        results = runTest(test_param_grid[2]['K'], e, (x_train, x_test), (y_train, y_test), (x, y), v['shape'], model_param_grid[key], key, (False, True, False))
        results = results['epsilon']
        results['dataset'] = key
        results['Epsilon'] = e

        intermediate.append(results)
    
    metrics = pd.DataFrame(intermediate)
    metrics.to_csv(pathlib.PurePath(res, 'strategy2_{}.csv'.format(key)))
    print("Test 2 Completed Successfully for {}".format(key))

Loading Partitions for CIFAR10 dataset with 1000 clusters
Running Test 2 on CIFAR10...


 25%|██▌       | 1/4 [13:51<41:33, 831.22s/it]

Accuracy on Epsilon_Neighbourhood : 0.3029


 50%|█████     | 2/4 [27:29<27:27, 823.90s/it]

Accuracy on Epsilon_Neighbourhood : 0.2994


...

Accuracy on Epsilon_Neighbourhood : 0.1164


d:\CONDA\envs\tfgpu\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 75%|███████▌  | 3/4 [48:53<16:22, 982.78s/it]

Accuracy on Epsilon_Neighbourhood : 0.1081


100%|██████████| 4/4 [1:07:53<00:00, 1018.43s/it]

Accuracy on Epsilon_Neighbourhood : 0.1006
Test 2 Completed Successfully for CIFAR100
